In [ ]:
import os as os
import numpy as np
import pandas as pd
import polars as pl
import tqdm
import slippi as slp
from joblib import Parallel, delayed
from multiprocessing import Manager

<h2> Preliminary Functions </h2>

We use these functions to one-hot encode the button bitmask and get the frame data for a given port number and frames object.

In [ ]:
# Set the number of time steps in the model inputs
frames_per_input = 60 * 12  # 12 seconds of gameplay

# Function to one-hot encode controller bitmask
def one_hot_encode(bitmask):
    labels = ['DPAD_LEFT', 'DPAD_RIGHT', 'DPAD_DOWN', 'DPAD_UP', 'Z', 'R', 'L', 'A', 'B', 'X', 'Y', 'START']
    encoded_values = [1, 2, 4, 8, 16, 32, 64, 256, 512, 1024, 2048, 4096]

    # Create a dictionary mapping labels to their encoded values
    label_to_value = dict(zip(labels, encoded_values))

    # Initialize a list to store the one-hot encoded values
    one_hot_encoded = [0] * len(labels)

    # Iterate through labels and set the corresponding one-hot encoded value
    for label, value in label_to_value.items():
        if bitmask & value:
            one_hot_encoded[labels.index(label)] = 1

    return one_hot_encoded

# Function to get frame data for a given set of frames and port
def get_frame_inputs(frames, port):
    sheik_inputs = np.empty((len(frames)-300, 18))  # Initialize an empty NumPy array    
    for i, frame in enumerate(frames[300: ]):   # Takes all the frames, skipping the first 5 seconds.
        buttons = one_hot_encode(frame.ports[port].leader.pre.buttons.physical.value)
        j_x = frame.ports[port].leader.pre.joystick.x
        j_y = frame.ports[port].leader.pre.joystick.y
        c_x = frame.ports[port].leader.pre.cstick.x
        c_y = frame.ports[port].leader.pre.cstick.y
        t_l = frame.ports[port].leader.pre.triggers.physical.l
        t_r = frame.ports[port].leader.pre.triggers.physical.r

        frame_data = buttons + [j_x, j_y, c_x, c_y, t_l, t_r]
        sheik_inputs[i] = frame_data

    return sheik_inputs





In [ ]:
# # Function to process a single SLP file and append to shared lists
# def process_slp_file(slp_file, dataset_path, time_series_list, label_list, ids):
#     try:
#         file_path = os.path.join(dataset_path, slp_file)
#         game = slp.Game(file_path)
#         frames = game.frames

#         if len(frames) < 300 + frames_per_input:  # Ignore games that are <3600 frames (i.e. <60 seconds)
#             return

#         # List occupied ports
#         occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
#         port_1 = occupied_ports[0]
#         port_2 = occupied_ports[1]

#         if len(occupied_ports) > 2:  # Ignore games that aren't singles
#             return

#         # Determine characters playing
#         port_1_character = game.start.players[port_1].character.name
#         port_2_character = game.start.players[port_2].character.name

#         frame_data = get_frame_data(frames, port_1)
#         time_series_list.append(frame_data)
#         label_list.append(1 if port_1_character == 'SHEIK' else 0)
#         ids.append(slp_file)
#         frame_data = get_frame_data(frames, port_2)
#         time_series_list.append(frame_data)
#         label_list.append(1 if port_2_character == 'SHEIK' else 0)
#         ids.append(slp_file)
#     except Exception as e:
#         print(f"Error processing {slp_file}: {str(e)}")

In [ ]:
        
# Set your dataset_path and frames_per_input
# dataset_path = './Slippi_Public_Dataset_v3/'

# slp_files = [file for file in os.listdir(dataset_path)]
# # print(len(slp_files))

# # Create polars data frame to store results
# game_data_df = pl.DataFrame(
#     {
#         "TimeSeries": [pl.col(pl.col([1, 2, 3])), pl.col(pl.col([4, 5, 6]))],
#         "Label": [pl.col(0), pl.col(1)],
#         "FName": [pl.col("file1.slp"), pl.col("file2.slp")]
#     }
# )

# Use joblib to parallelize processing of SLP files
# Parallel(n_jobs=-1, verbose=10)(delayed(process_slp_file)(slp_file, dataset_path, time_series_list, label_list, ids) for slp_file in tqdm.tqdm(slp_files))

# Create a DataFrame from the results
# df = pd.DataFrame({"TimeSeries": list(time_series_list), "Label": list(label_list), "FName": list(ids)})


# df.sort_values(by=['FName','Label'],inplace=True)
# df.reset_index(drop=True)
# print(df)

In [ ]:
# Set the number of time steps in the model inputs
frames_per_input = 60 * 12  # 12 seconds of gameplay

dataset_path = './Slippi_Public_Dataset_v3/'

# List of file names
slp_files = [file for file in os.listdir(dataset_path)]

slp_file = slp_files[10]

# A single file path
file_path = os.path.join(dataset_path, slp_file)


# game_data_df = 

# Function to process a single SLP file and append to shared polars df
def process_slp_file(slp_file, dataset_path,game_data_df,schema):#,df):#, time_series_list, label_list, ids):
    try:
        file_path = os.path.join(dataset_path, slp_file)
        game = slp.Game(file_path)
        frames = game.frames

        if len(frames) < 300 + frames_per_input:  # Ignore games that are <3600 frames (i.e. <60 seconds)
            return

        # List occupied ports
        occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
        port_1 = occupied_ports[0]
        port_2 = occupied_ports[1]

        if len(occupied_ports) > 2:  # Ignore games that aren't singles
            return
        
        # port_1_frame_
        port_1_frame_inputs = get_frame_inputs(frames, port_1)
        port_2_frame_inputs = get_frame_inputs(frames, port_2)
        
        game_data_dict = {
            # Start (Game Data)
            'file': slp_file,
            # 'is_teams': game.start.stage.is_teams, # bool
            # 'players': game.start.stage.players, #tuple
            # 'random_seed':game.start.stage.random_seed # int
            # 'slippi': game.start.stage.slippi #Slippi
            'stage_name': game.start.stage.name,
            'stage_value': game.start.stage.value,
            'is_pal': game.start.is_pal,
            'is_frozen_ps': game.start.is_frozen_ps,
            # Player 1 Data
            'port_1': port_1,
            'port_1_character_name': game.start.players[port_1].character.name,
            'port_1_character_value': game.start.players[port_1].character.value,
            'port_1_type_name': game.start.players[port_1].type.name,
            'port_1_type_value': game.start.players[port_1].type.value,
            'port_1_stocks': game.start.players[port_1].stocks,
            'port_1_costume': game.start.players[port_1].costume,
            # 'port_1_team': game.start.players[port_1].team # teem|None
            'port_1_ucf_dash_back_name': game.start.players[port_1].ucf.dash_back.name,
            'port_1_ucf_dash_back_value': game.start.players[port_1].ucf.dash_back.value,
            'port_1_ucf_shield_drop_name': game.start.players[port_1].ucf.shield_drop.name,
            'port_1_ucf_shield_drop_value': game.start.players[port_1].ucf.shield_drop.value,
            'port_1_tag': game.start.players[port_1].tag,
            # Player 2 Data
            'port_2': port_2,
            'port_2_character_name': game.start.players[port_2].character.name,
            'port_2_character_value': game.start.players[port_2].character.value,
            'port_2_type_name': game.start.players[port_2].type.name,
            'port_2_type_value': game.start.players[port_2].type.value,
            'port_2_stocks': game.start.players[port_2].stocks,
            'port_2_costume': game.start.players[port_2].costume,
            'port_2_ucf_dash_back_name': game.start.players[port_2].ucf.dash_back.name,
            'port_2_ucf_dash_back_value': game.start.players[port_2].ucf.dash_back.value,
            'port_2_ucf_shield_drop_name': game.start.players[port_2].ucf.shield_drop.name,
            'port_2_ucf_shield_drop_value': game.start.players[port_2].ucf.shield_drop.value,
            'port_2_tag': game.start.players[port_2].tag,
            # End
            'method_how_game_ended': game.end.method, #string
            # Player 1 Pre Frame data
            'port_1_pre_state_value': [np.array([frame.ports[port_1].leader.pre.state.value if frame.ports[port_1].leader.pre.state else None for frame in frames[300:]])], #object
            'port_1_pre_state_name': [np.array([frame.ports[port_1].leader.pre.state.name if frame.ports[port_1].leader.pre.state else None for frame in frames[300:]])], #object
            'port_1_pre_position_x': [np.array([frame.ports[port_1].leader.pre.position.x for frame in frames[300:]])], #object
            'port_1_pre_position_y': [np.array([frame.ports[port_1].leader.pre.position.y for frame in frames[300:]])], #object
            'port_1_pre_direction': [np.array([frame.ports[port_1].leader.pre.direction for frame in frames[300:]])], #object
            'port_1_pre_damage':[np.array([frame.ports[port_1].leader.pre.damage for frame in frames[300:]])], #object
            # Player 1 Frame pre Inputs
            'port_1_DPAD_LEFT': [port_1_frame_inputs[:, 0]],
            'port_1_DPAD_RIGHT': [port_1_frame_inputs[:, 1]],
            'port_1_DPAD_DOWN': [port_1_frame_inputs[:, 2]],
            'port_1_DPAD_UP': [port_1_frame_inputs[:, 3]],
            'port_1_Z': [port_1_frame_inputs[:, 4]],
            'port_1_R': [port_1_frame_inputs[:, 5]],
            'port_1_L': [port_1_frame_inputs[:, 6]],
            'port_1_A': [port_1_frame_inputs[:, 7]],
            'port_1_B': [port_1_frame_inputs[:, 8]],
            'port_1_X': [port_1_frame_inputs[:, 9]],
            'port_1_Y': [port_1_frame_inputs[:, 10]],
            'port_1_START': [port_1_frame_inputs[:, 11]],
            'port_1_J_X': [port_1_frame_inputs[:, 12]],
            'port_1_J_Y': [port_1_frame_inputs[:, 13]],
            'port_1_C_X': [port_1_frame_inputs[:, 14]],
            'port_1_C_Y': [port_1_frame_inputs[:, 15]],
            'port_1_T_L': [port_1_frame_inputs[:, 16]],
            'port_1_T_R': [port_1_frame_inputs[:, 17]],
            # Player 1 post frame data
            'port_1_post_character_value': [np.array([frame.ports[port_1].leader.post.character.value if frame.ports[port_1].leader.post.character else None for frame in frames[300:]])], #object
            'port_1_post_character_name': [np.array([frame.ports[port_1].leader.post.character.name if frame.ports[port_1].leader.post.character else None for frame in frames[300:]])], #object
            'port_1_post_state_value': [np.array([frame.ports[port_1].leader.post.state.value if frame.ports[port_1].leader.post.state else None for frame in frames[300:]])], #object
            'port_1_post_state_name': [np.array([frame.ports[port_1].leader.post.state.name if frame.ports[port_1].leader.post.state else None for frame in frames[300:]])], #object
            'port_1_post_position': [np.array([frame.ports[port_1].leader.post.position.y for frame in frames[300:]])], #object
            'port_1_post_direction': [np.array([frame.ports[port_1].leader.post.direction for frame in frames[300:]])], #object
            'port_1_post_damage': [np.array([frame.ports[port_1].leader.post.damage for frame in frames[300:]])], #object
            'port_1_post_shield': [np.array([frame.ports[port_1].leader.post.shield for frame in frames[300:]])], #object
            'port_1_post_stocks': [np.array([frame.ports[port_1].leader.post.stocks for frame in frames[300:]])], #object
            'port_1_post_last_attack_landed_value': [np.array([frame.ports[port_1].leader.post.last_attack_landed.value if frame.ports[port_1].leader.post.last_attack_landed else None for frame in frames[300:]])], #object
            'port_1_post_last_attack_landed_name': [np.array([frame.ports[port_1].leader.post.last_attack_landed.name if frame.ports[port_1].leader.post.last_attack_landed else None for frame in frames[300:]])], #object
            # 'port_1_post_last_hit_by': [np.array([frame.ports[port_1].leader.post.last_hit_by for frame in frames[300:]])], #object
            'port_1_post_combo_count': [np.array([frame.ports[port_1].leader.post.combo_count for frame in frames[300:]])], #object
            # 'port_1_post_state_age': [np.array([frame.ports[port_1].leader.post.state_age for frame in frames[300:]])], #object
            'port_1_post_flags_value': [np.array([frame.ports[port_1].leader.post.flags.value for frame in frames[300:]])], #object
            'port_1_post_hit_stun': [np.array([frame.ports[port_1].leader.post.hit_stun for frame in frames[300:]])], #object ??????
            'port_1_post_airborne': [np.array([frame.ports[port_1].leader.post.airborne for frame in frames[300:]])], #object
            'port_1_post_ground': [np.array([frame.ports[port_1].leader.post.ground for frame in frames[300:]])], #object
            'port_1_post_jumps': [np.array([frame.ports[port_1].leader.post.jumps for frame in frames[300:]])], #object
            'port_1_post_l_cancel': [np.array([frame.ports[port_1].leader.post.l_cancel for frame in frames[300:]])], #object
             # Player 2 Pre Frame data
            'port_2_pre_state_value': [np.array([frame.ports[port_2].leader.pre.state.value if frame.ports[port_2].leader.pre.state else None for frame in frames[300:]])], #object
            'port_2_pre_state_name': [np.array([frame.ports[port_2].leader.pre.state.name if frame.ports[port_2].leader.pre.state else None for frame in frames[300:]])], #object
            'port_2_pre_position_x': [np.array([frame.ports[port_2].leader.pre.position.x for frame in frames[300:]])], #object
            'port_2_pre_position_y': [np.array([frame.ports[port_2].leader.pre.position.y for frame in frames[300:]])], #object
            'port_2_pre_direction': [np.array([frame.ports[port_2].leader.pre.direction for frame in frames[300:]])], #object
            'port_2_pre_damage':[np.array([frame.ports[port_2].leader.pre.damage for frame in frames[300:]])], #object
            # Player 2 Frame pre Inputs
            'port_2_DPAD_LEFT': [port_2_frame_inputs[:, 0]],
            'port_2_DPAD_RIGHT': [port_2_frame_inputs[:, 1]],
            'port_2_DPAD_DOWN': [port_2_frame_inputs[:, 2]],
            'port_2_DPAD_UP': [port_2_frame_inputs[:, 3]],
            'port_2_Z': [port_2_frame_inputs[:, 4]],
            'port_2_R': [port_2_frame_inputs[:, 5]],
            'port_2_L': [port_2_frame_inputs[:, 6]],
            'port_2_A': [port_2_frame_inputs[:, 7]],
            'port_2_B': [port_2_frame_inputs[:, 8]],
            'port_2_X': [port_2_frame_inputs[:, 9]],
            'port_2_Y': [port_2_frame_inputs[:, 10]],
            'port_2_START': [port_2_frame_inputs[:, 11]],
            'port_2_J_X': [port_2_frame_inputs[:, 12]],
            'port_2_J_Y': [port_2_frame_inputs[:, 13]],
            'port_2_C_X': [port_2_frame_inputs[:, 14]],
            'port_2_C_Y': [port_2_frame_inputs[:, 15]],
            'port_2_T_L': [port_2_frame_inputs[:, 16]],
            'port_2_T_R': [port_2_frame_inputs[:, 17]],
            # Player 2 post frame data
            'port_2_post_character_value': [np.array([frame.ports[port_2].leader.post.character.value if frame.ports[port_2].leader.post.character else None for frame in frames[300:]])], #object
            'port_2_post_character_name': [np.array([frame.ports[port_2].leader.post.character.name if frame.ports[port_2].leader.post.character else None for frame in frames[300:]])], #object
            'port_2_post_state_value': [np.array([frame.ports[port_2].leader.post.state.value if frame.ports[port_2].leader.post.state else None for frame in frames[300:]])], #object
            'port_2_post_state_name': [np.array([frame.ports[port_2].leader.post.state.name if frame.ports[port_2].leader.post.state else None for frame in frames[300:]])], #object
            'port_2_post_position': [np.array([frame.ports[port_2].leader.post.position.y for frame in frames[300:]])], #object
            'port_2_post_direction': [np.array([frame.ports[port_2].leader.post.direction for frame in frames[300:]])], #object
            'port_2_post_damage': [np.array([frame.ports[port_2].leader.post.damage for frame in frames[300:]])], #object
            'port_2_post_shield': [np.array([frame.ports[port_2].leader.post.shield for frame in frames[300:]])], #object
            'port_2_post_stocks': [np.array([frame.ports[port_2].leader.post.stocks for frame in frames[300:]])], #object
            'port_2_post_last_attack_landed_value': [np.array([frame.ports[port_2].leader.post.last_attack_landed.value if frame.ports[port_2].leader.post.last_attack_landed else None for frame in frames[300:]])], #object
            'port_2_post_last_attack_landed_name': [np.array([frame.ports[port_2].leader.post.last_attack_landed.name if frame.ports[port_2].leader.post.last_attack_landed else None for frame in frames[300:]])], #object
            # 'port_2_post_last_hit_by': [np.array([frame.ports[port_2].leader.post.last_hit_by for frame in frames[300:]])], #object
            'port_2_post_combo_count': [np.array([frame.ports[port_2].leader.post.combo_count for frame in frames[300:]])], #object
            # 'port_2_post_state_age': [np.array([frame.ports[port_2].leader.post.state_age for frame in frames[300:]])], #object
            'port_2_post_flags_value': [np.array([frame.ports[port_2].leader.post.flags.value for frame in frames[300:]])], #object
            'port_2_post_hit_stun': [np.array([frame.ports[port_2].leader.post.hit_stun for frame in frames[300:]])], #object ??????
            'port_2_post_airborne': [np.array([frame.ports[port_2].leader.post.airborne for frame in frames[300:]])], #object
            'port_2_post_ground': [np.array([frame.ports[port_2].leader.post.ground for frame in frames[300:]])], #object
            'port_2_post_jumps': [np.array([frame.ports[port_2].leader.post.jumps for frame in frames[300:]])], #object
            'port_2_post_l_cancel': [np.array([frame.ports[port_2].leader.post.l_cancel for frame in frames[300:]])], #object
            #Item
            # 'item_':[np.array([frame.Item.type.value for frame in frames[300:]])]# I CANT FIGURE THIS ONE OUT
            # 'item_state':[np.array([frame.Item.state for frame in frames[300:]])],# object
            # 'item_direction':[np.array([frame.Item.direction for frame in frames[300:]])],# object
            # 'item_velocity_x':[np.array([frame.Item.velocity.x for frame in frames[300:]])],# object
            # 'item_velocity_y':[np.array([frame.Item.velocity.y for frame in frames[300:]])],# object
            # 'item_position_x':[np.array([frame.Item.position.x for frame in frames[300:]])],# object
            # 'item_position_y':[np.array([frame.Item.position.y for frame in frames[300:]])],# object
            # 'item_damage':[np.array([frame.Item.damage for frame in frames[300:]])],# object
            # 'item_timer':[np.array([frame.Item.timer for frame in frames[300:]])],# object
            # 'item_spawn_id':[np.array([frame.Item.spawn_id for frame in frames[300:]])],# object
            
        }
    
        # game_data_dict['port_1_inputs'] = [get_frame_inputs(frames, port_1)]
        # game_data_dict['port_2_inputs'] = [get_frame_inputs(frames, port_2)]
        # Convert the game_data_dict to a Polars DataFrame without port_1_inputs and port_2_inputs
        print(game_data_dict)
        return game_data_dict
    except Exception as e:
        print(f"Error processing {slp_file}: {str(e)}")
        


    
# Create an empty Polars DataFrame with the desired schema
schema =    [
    ('file', pl.String),
    ('stage_name', pl.String),
    ('stage_value', pl.Int8),
    ('is_pal', pl.Boolean),
    ('is_frozen_ps', pl.Boolean),
    # Player 1 Data
    ('port_1', pl.Int32),
    ('port_1_character_name', pl.String),
    ('port_1_character_value', pl.Int32),
    ('port_1_type_name', pl.String),
    ('port_1_type_value', pl.Boolean),
    ('port_1_stocks', pl.Int32),
    ('port_1_costume', pl.Int32),
    ('port_1_ucf_dash_back_name', pl.String),
    ('port_1_ucf_dash_back_value', pl.Int32),
    ('port_1_ucf_shield_drop_name', pl.String),
    ('port_1_ucf_shield_drop_value', pl.Int32),
    ('port_1_tag', pl.String),
    # Player 2 Data
    ('port_2', pl.Int32),
    ('port_2_character_name', pl.String),
    ('port_2_character_value', pl.Int32),
    ('port_2_type_name', pl.String),
    ('port_2_type_value', pl.Boolean),
    ('port_2_stocks', pl.Int32),
    ('port_2_costume', pl.Int32),
    ('port_2_ucf_dash_back_name', pl.String),
    ('port_2_ucf_dash_back_value', pl.Int32),
    ('port_2_ucf_shield_drop_name', pl.String),
    ('port_2_ucf_shield_drop_value', pl.Int32),
    ('port_2_tag', pl.String),
     # End
    ('method_how_game_ended',pl.String),
    #  # Player 1 Pre Frame data
    ('port_1_pre_state_value',pl.Object),
    ('port_1_pre_state_name',pl.Object),
    ('port_1_pre_position_x',pl.Object),
    ('port_1_pre_position_y',pl.Object),
    ('port_1_pre_direction',pl.Object),
    ('port_1_pre_damage',pl.Object),
        # pre Input Data for Player 1
    ('port_1_DPAD_LEFT', pl.Object),
    ('port_1_DPAD_RIGHT', pl.Object),
    ('port_1_DPAD_DOWN', pl.Object),
    ('port_1_DPAD_UP', pl.Object),
    ('port_1_Z', pl.Object),
    ('port_1_R', pl.Object),
    ('port_1_L', pl.Object),
    ('port_1_A', pl.Object),
    ('port_1_B', pl.Object),
    ('port_1_X', pl.Object),
    ('port_1_Y', pl.Object),
    ('port_1_START', pl.Object),
    ('port_1_J_X', pl.Object),
    ('port_1_J_Y', pl.Object),
    ('port_1_C_X', pl.Object),
    ('port_1_C_Y', pl.Object),
    ('port_1_T_L', pl.Object),
    ('port_1_T_R', pl.Object),
    # Player 1 post frame data
    ('port_1_post_character_value', pl.Object),
    ('port_1_post_character_name', pl.Object),
    ('port_1_post_state_value', pl.Object),
    ('port_1_post_state_name', pl.Object),
    ('port_1_post_position', pl.Object),
    ('port_1_post_direction', pl.Object),
    ('port_1_post_damage', pl.Object),
    ('port_1_post_shield', pl.Object),
    ('port_1_post_stocks', pl.Object),
    ('port_1_post_last_attack_landed_value', pl.Object),
    ('port_1_post_last_attack_landed_name', pl.Object),
    ('port_1_post_combo_count', pl.Object),
    ('port_1_post_flags_value', pl.Object),
    ('port_1_post_hit_stun', pl.Object),
    ('port_1_post_airborne', pl.Object),
    ('port_1_post_ground', pl.Object),
    ('port_1_post_jumps', pl.Object),
    ('port_1_post_l_cancel', pl.Object),
    # Player 1 Pre Frame data
    ('port_2_pre_state_value',pl.Object),
    ('port_2_pre_state_name',pl.Object),
    ('port_2_pre_position_x',pl.Object),
    ('port_2_pre_position_y',pl.Object),
    ('port_2_pre_direction',pl.Object),
    ('port_2_pre_damage',pl.Object),
    # pre Input Data for Player 1
    ('port_2_DPAD_LEFT', pl.Object),
    ('port_2_DPAD_RIGHT', pl.Object),
    ('port_2_DPAD_DOWN', pl.Object),
    ('port_2_DPAD_UP', pl.Object),
    ('port_2_Z', pl.Object),
    ('port_2_R', pl.Object),
    ('port_2_L', pl.Object),
    ('port_2_A', pl.Object),
    ('port_2_B', pl.Object),
    ('port_2_X', pl.Object),
    ('port_2_Y', pl.Object),
    ('port_2_START', pl.Object),
    ('port_2_J_X', pl.Object),
    ('port_2_J_Y', pl.Object),
    ('port_2_C_X', pl.Object),
    ('port_2_C_Y', pl.Object),
    ('port_2_T_L', pl.Object),
    ('port_2_T_R', pl.Object),
    # Player 1 post frame data
    ('port_2_post_character_value', pl.Object),
    ('port_2_post_character_name', pl.Object),
    ('port_2_post_state_value', pl.Object),
    ('port_2_post_state_name', pl.Object),
    ('port_2_post_position', pl.Object),
    ('port_2_post_direction', pl.Object),
    ('port_2_post_damage', pl.Object),
    ('port_2_post_shield', pl.Object),
    ('port_2_post_stocks', pl.Object),
    ('port_2_post_last_attack_landed_value', pl.Object),
    ('port_2_post_last_attack_landed_name', pl.Object),
    ('port_2_post_combo_count', pl.Object),
    ('port_2_post_flags_value', pl.Object),
    ('port_2_post_hit_stun', pl.Object),
    ('port_2_post_airborne', pl.Object),
    ('port_2_post_ground', pl.Object),
    ('port_2_post_jumps', pl.Object),
    ('port_2_post_l_cancel', pl.Object),

    # Item (You can uncomment and add these fields if needed)
    # ('item_type_value', pl.Object),
    # ('item_state', pl.Object),
    # ('item_direction', pl.Object),
    # ('item_velocity_x', pl.Object),
    # ('item_velocity_y', pl.Object),
    # ('item_position_x', pl.Object),
    # ('item_position_y', pl.Object),
    # ('item_damage', pl.Object),
    # ('item_timer', pl.Object),
    # ('item_spawn_id', pl.Object),
]




game_data_df = pl.DataFrame([], schema=schema)
game_data_dict = process_slp_file(slp_file, dataset_path,game_data_df,schema)
print(game_data_dict)
game_data_new_row = pl.DataFrame(game_data_dict,schema=schema)
game_data_df.vstack(game_data_new_row ,in_place = True)
print(game_data_df)

# print(game_data_df)

# game_data_df

In [ ]:
file_path = os.path.join(dataset_path, slp_file)
game = slp.Game(file_path)
frames = game.frames

# if len(frames) < 300 + frames_per_input:  # Ignore games that are <3600 frames (i.e. <60 seconds)
#     return

# List occupied ports
occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
port_1 = occupied_ports[0]
port_2 = occupied_ports[1]
frame = frames[100]
# frames[100].Item.type
# if len(occupied_ports) > 2:  # Ignore games that aren't singles
#     return

# port_1_frame_
# port_1_frame_inputs = get_frame_inputs(frames, port_1)
# port_2_frame_inputs = get_frame_inputs(frames, port_2)

# arr = np.array([frame.Item.name for frame in frames[300:]])
# print(arr)
# print(np.unique(arr))

# print(frames[100].Item.type)
# print(np.array([frame.ports[port_1].leader.post.state.value for frame in frames[300:]]))
# print(np.array([frame.ports[port_1].leader.post.state.name for frame in frames[300:]]))
np.array([frame.Item.type.value for frame in frames[300:]]

In [ ]:
# Define a mapping from item type to item name
item_type_to_name = {
    0: "None",
    1: "Blast Box",
    2: "Capsule",
    # Add more mappings for other item types here
}

# Iterate through the frames and extract item names
for frame in frames[300:]:
    if frame.Item:
        item_type = frame.Item.type
        item_name = item_type_to_name.get(item_type)
        # item_names.append(item_name)

# Convert the list of item names to a NumPy array
# item_names_array = np.array(item_names)

# Print the unique item names
# unique_item_names = np.unique(item_names_array)
# print(unique_item_names)

# frame.ports[port_1].leader.pre.state
np.array([frame.ports[port_2].leader.pre.damage for frame in frames[300:]])